### 1 - Import test data

In [3]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2024-10-1", end="2024-11-1", interval='5M')
dataF.columns = dataF.columns.get_level_values('Price')
dataF.iloc[:,:]
# dataF.Open.iloc

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Datetime,,,,,,
2024-10-01 00:00:00,1.114455,1.114455,1.114455,1.114455,1.114455,0
2024-10-01 00:05:00,1.114330,1.114330,1.114579,1.114330,1.114455,0
2024-10-01 00:10:00,1.114455,1.114455,1.114455,1.114455,1.114455,0
2024-10-01 00:15:00,1.114579,1.114579,1.114579,1.114455,1.114455,0
2024-10-01 00:20:00,1.114455,1.114455,1.114579,1.114330,1.114579,0
...,...,...,...,...,...,...
2024-10-31 23:35:00,1.088732,1.088732,1.088850,1.088732,1.088732,0
2024-10-31 23:40:00,1.088850,1.088850,1.088850,1.088850,1.088850,0
2024-10-31 23:45:00,1.088732,1.088732,1.088850,1.088732,1.088732,0


### 2 - Define your signal function - Where you put your strat

In [ ]:
#Extras
# # ANSI escape sequences for text color
# print("\033[1;31mThis is red text")    # Red text
# print("\033[1;32mThis is green text")  # Green text
# print("\033[1;33mThis is yellow text") # Yellow text
# print("\033[0mThis is default text")   # Reset to default color

def signal_generator(df):#Engulfing pattern
    # Extract scalar values for open and close prices
    # print("signal_generator running")
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    prevOpen = df.Open.iloc[-2]
    prevClose = df.Close.iloc[-2]
    # print("\033[1;33m"+ "| open: " + str(open) + " | close: " + str(close) + " | prevOpen: " + str(prevOpen) + " | prevClose: " + str(prevClose) + "\033[0m", end="")

    # Bearish Pattern
    if ( (open > close) and (prevOpen < prevClose) and 
        (close < prevOpen) and (open >= prevClose) ):
        # print(" _-Bearish Pattern-_")
        return 1

    # Bullish Pattern
    elif ( (open < close) and (prevOpen > prevClose) and 
          (close > prevOpen) and (open <= prevClose) ):
        # print(" _-Bearish Pattern-_")
        return 2

    # No clear pattern
    else:
        # print(" _-Bearish Pattern-_")
        return 0


#Test dataframe self

# Generate signals
signal = [0]  # First row has no signal
for i in range(1, len(dataF)):
    # Slice two rows for signal generation
    df = dataF.iloc[i-1:i+1]
    signal.append(signal_generator(df))

print("\033[1;32m___Signal generating DONE___\033[0m", )#Set text to green and reset to default

# Assign the signals to the DataFrame
dataF["signal"] = signal

# Check signal distribution
print(dataF["signal"].value_counts())
dataF.iloc[:,:]

#Write current dataF to excel spreadsheet
dataF.to_excel('dataF_1.xlsx',header=True, index=True)


___Signal generating DONE___
signal
0      6356
999      88
2        84
Name: count, dtype: int64


In [4]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

0    3527
2     178
1     172
Name: signal, dtype: int64

### 3 - Connect to the market and execute trades

In [5]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [6]:
from config import access_token, accountID
def get_candles(n):
    #access_token='XXXXXXX'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


True
True
True


In [12]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [ ]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()